In [1]:
# from espnapi.espn_api.football import League
from espn_api.football import League
import pandas as pd
import plotly.express as px
# Init

#fan's - 42936131
#mine - 829963546
league_id = 42936131
year = 2022
league = League(league_id=league_id, year=year)



## Lets get the draft picks from each team

In [2]:
df = pd.DataFrame()

teams = []
players = []
id = []
round_num = []
round_pick = []


for pick in league.draft:
    teams.append(pick.team.team_name)
    players.append(pick.playerName)
    round_num.append(pick.round_num)
    round_pick.append(pick.round_pick)
    id.append(pick.playerId)
    
df['Team'] = teams
df['Player'] = players
df['ID'] = id
df['RoundPick'] = round_pick
df['RoundNum'] = round_num
df.to_csv('Data/team_players.csv')
df.head(10)

,Team,Player,ID,RoundPick,RoundNum
0,JTT 4EVR,Jonathan Taylor,4242335,1,1
1,Dessert Desperadoes,Austin Ekeler,3068267,2,1
2,3 PP Wine,Christian McCaffrey,3117251,3,1
3,Sunday Funday,Cooper Kupp,2977187,4,1
4,Devon Allen’s Hurdle crew,Derrick Henry,3043078,5,1
5,Trash Trash,Najee Harris,4241457,6,1
6,Dakcidental Thumblers,Ja'Marr Chase,4362628,7,1
7,Hot Chubb Time Machine 2,Justin Jefferson,4262921,8,1
8,RightInThe McManus,Dalvin Cook,3116593,9,1
9,Deshaunorable Discharge,Joe Mixon,3116385,10,1


# Get projected points

In [3]:
players = []
expected = []
actual = []
rank = []
nfl_team = []
for week in range(10):
    league.current_week = week
    for team in league.teams:
        for player in team.roster:
            players.append(player.name)
            expected.append(player.projected_total_points)
            actual.append(player.total_points)
            rank.append(player.posRank)
            nfl_team.append(player.proTeam)

points_df = pd.DataFrame()
points_df['Player'] = players
points_df['Expected_Points'] = expected
points_df['Actual_Points'] = actual
points_df['Rank'] = rank
points_df['NFLTeam'] = nfl_team



# Calculate Differences

In [4]:
points_df['Difference'] = points_df['Actual_Points'] - points_df['Expected_Points']
points_df = points_df.sort_values('Difference', ascending = False)

# Merge dataframes

In [5]:
combined_df = pd.merge(df, points_df, on = 'Player')
combined_df.head(10)

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
1,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
2,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
3,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
4,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
5,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
6,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
7,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
8,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59
9,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59


In [6]:
#normalize
combined_df['Normalized_Difference'] = combined_df['Difference'] /combined_df['Difference'].abs().max()
  

In [7]:
combined_df

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference,Normalized_Difference
0,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59,-0.865983
1,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59,-0.865983
2,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59,-0.865983
3,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59,-0.865983
4,JTT 4EVR,Jonathan Taylor,4242335,1,1,316.99,146.4,26,IND,-170.59,-0.865983
...,...,...,...,...,...,...,...,...,...,...,...
1445,JTT 4EVR,Rondale Moore,4372485,12,16,130.69,87.9,72,ARI,-42.79,-0.217219
1446,JTT 4EVR,Rondale Moore,4372485,12,16,130.69,87.9,72,ARI,-42.79,-0.217219
1447,JTT 4EVR,Rondale Moore,4372485,12,16,130.69,87.9,72,ARI,-42.79,-0.217219
1448,JTT 4EVR,Rondale Moore,4372485,12,16,130.69,87.9,72,ARI,-42.79,-0.217219


# Now get their actual players

In [8]:
teams = []
players = []
projected = []
actual = []
stats = []
ranks = []
weeks = []
current_df = pd.DataFrame()
for i in range(1, 13):
    league.load_roster_week(i)
    for team in league.teams:
        for player in team.roster:
            weeks.append(i)
            teams.append(team.team_name)
            players.append(player.name)
            projected.append(player.projected_total_points)
            actual.append(player.total_points)
            ranks.append(player.posRank)

current_df['Team'] = teams
current_df['Player'] = players
current_df['Week'] = weeks
current_df['Rank'] = ranks
current_df['Expected_Points'] = projected
current_df['Actual_Points'] = actual
current_df['Week'] = weeks
current_df['Difference'] = current_df['Actual_Points'] - current_df['Expected_Points']

In [9]:
current_df.to_csv('Data/current_players.csv')

In [10]:
combined_df.groupby('Team').mean()

,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,Difference,Normalized_Difference
Team,,,,,,,,
3 PP Wine,3.031240e+06,6.230769,8.076923,193.156923,149.795385,36.076923,-43.361538,-0.220121
Corduroy Patterson,2.376335e+06,6.500000,7.928571,194.495714,154.610000,26.785714,-39.885714,-0.202476
Country Road Take Mahomes,3.318633e+06,6.500000,6.833333,194.146667,159.995000,21.583333,-34.151667,-0.173368
Dakcidental Thumblers,3.575556e+06,6.461538,7.846154,200.966154,162.172308,22.384615,-38.793846,-0.196933
Deshaunorable Discharge,3.853376e+06,7.200000,6.400000,205.381000,173.120000,23.900000,-32.261000,-0.163770
Dessert Desperadoes,3.337283e+06,6.500000,7.642857,187.500000,155.568571,26.785714,-31.931429,-0.162097
Devon Allen’s Hurdle crew,3.065685e+06,6.363636,6.545455,201.901818,194.130909,12.636364,-7.770909,-0.039448
Hot Chubb Time Machine 2,3.425774e+06,6.666667,6.777778,191.817778,176.817778,18.444444,-15.000000,-0.076146
JTT 4EVR,3.234569e+06,6.133333,8.533333,198.615333,154.733333,28.666667,-43.882000,-0.222763


In [11]:
print('Done')

Done


,Team,Player,Week,Rank,Expected_Points,Actual_Points,Difference
33,Sunday Funday,Cooper Kupp,1,13,312.91,201.4,-111.51
228,Sunday Funday,Cooper Kupp,2,13,312.91,201.4,-111.51
424,Sunday Funday,Cooper Kupp,3,13,312.91,201.4,-111.51
620,Sunday Funday,Cooper Kupp,4,13,312.91,201.4,-111.51
816,Sunday Funday,Cooper Kupp,5,13,312.91,201.4,-111.51
1012,Sunday Funday,Cooper Kupp,6,13,312.91,201.4,-111.51
1210,Sunday Funday,Cooper Kupp,7,13,312.91,201.4,-111.51
1408,Sunday Funday,Cooper Kupp,8,13,312.91,201.4,-111.51
1608,Sunday Funday,Cooper Kupp,9,13,312.91,201.4,-111.51
1808,Sunday Funday,Cooper Kupp,10,13,312.91,201.4,-111.51
